In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
!pip install apache_beam mwparserfromhell

In [ ]:
!pip install datasets

In [45]:
## public libraries
from datasets import load_dataset

## local libraries
from gpt_utils import (
    get_encoder_decoder_size,
    text_to_tv_tensors,
    get_batch
)
from attention_decoder import AttentionModule


In [2]:
wiki_raw = load_dataset("wikipedia", "20220301.simple")

Found cached dataset wikipedia (/Users/paul/.cache/huggingface/datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
wiki_raw

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 205328
    })
})

In [6]:
wiki_text_blob = '\n\n'.join(wiki_raw['train']['text'])
print(len(wiki_text_blob))
print()
print(wiki_text_blob[:1000])

215900536

April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.

April always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.

April's flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.

The Month 

April comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.

April begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other's last days are exactly 35 weeks (245 days) apart.

In common years, April starts on the same day of the week as October of the previous year, and in le

In [5]:
##When sorted, low-numbered codes seem to be more "normal"
normal_chars = ''.join(sorted(list(set(wiki_text_blob)))[0:97])
normal_chars

'\t\n !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~'

In [7]:
# 30 sec ish
wiki_text_blob_clean = ''.join([x for x in wiki_text_blob if x in normal_chars])

In [8]:
# Keep 99.7% of chars. great.
len(wiki_text_blob_clean)/len(wiki_text_blob)

0.997293827931951

In [25]:
encode, decode, vocab_size = get_encoder_decoder_size(wiki_text_blob_clean)
print(vocab_size)

97


In [26]:
train, validate = text_to_tv_tensors(wiki_text_blob_clean, encode)

torch.Size([215316272]) torch.int64
tensor([35, 82, 84, 75, 78,  2, 75, 85,  2, 86, 74, 71,  2, 72, 81, 87, 84, 86,
        74,  2, 79, 81, 80, 86, 74,  2, 81, 72,  2, 86, 74, 71,  2, 91, 71, 67,
        84,  2, 75, 80,  2, 86, 74, 71,  2, 44, 87, 78, 75, 67, 80,  2, 67, 80,
        70,  2, 41, 84, 71, 73, 81, 84, 75, 67, 80,  2, 69, 67, 78, 71, 80, 70,
        67, 84, 85, 14,  2, 67, 80, 70,  2, 69, 81, 79, 71, 85,  2, 68, 71, 86,
        89, 71, 71, 80,  2, 47, 67, 84, 69, 74])


In [47]:
m_attn = AttentionModule(vocab_size)

In [50]:
# optimizer = torch.optim.AdamW(m8_wiki.parameters(), lr=1e-3)
torch.manual_seed(42)

optimizer = torch.optim.AdamW(m_attn.parameters(), lr=1e-3)
BATCH_SIZE = 32

for epoch in range(1000):
  tx, ty = get_batch(train)

  _, loss = m_attn(tx, ty)
  optimizer.zero_grad(set_to_none=True)
  if epoch%100 == 0:
    print(epoch, loss)
  loss.backward()
  optimizer.step()

0 tensor(2.5938, grad_fn=<NllLossBackward>)
100 tensor(2.4575, grad_fn=<NllLossBackward>)
200 tensor(2.5328, grad_fn=<NllLossBackward>)
300 tensor(2.3907, grad_fn=<NllLossBackward>)
400 tensor(2.4390, grad_fn=<NllLossBackward>)
500 tensor(2.4191, grad_fn=<NllLossBackward>)
600 tensor(2.5385, grad_fn=<NllLossBackward>)
700 tensor(2.4829, grad_fn=<NllLossBackward>)
800 tensor(2.3916, grad_fn=<NllLossBackward>)
900 tensor(2.5243, grad_fn=<NllLossBackward>)


In [51]:
seed_raw = "To be or not to "
seed = torch.tensor(encode(seed_raw)).view(1,-1)

print(decode(m_attn.generate(seed, 1000)[0,]))

To be or not to thip cask RIst 193 205
 201953 pror ynspereanis. c Toustars anfercs Destttinings Mibeetgauet ohe he nasies  19" is al ins sp and birias Baus nthisidanaret Hapstt and modd they and opece eont Glatrsh, Epanacals Jucky

19
is beostizitirebus) He brafitarar acmes of futrsg.
thed Slony nose ahaus thing warves  greatson" in atsthoos: ale  w'irnctar ding an F he wnoly
 Fremip 19 dider, Fury ath wishel whithiriflis wingl wifth the myypersana.
 Ref Mavatos, and uso Bucknealie San gies the sos cat su., Noat couf Das"|216 T||||0|3)

|0,) ane Rif Morcen, cantlerian sononters, Domun wos gillsst. It Janmong a Fe bules the  uffam Nese nand, ame pantie (69).16, 1984 omec.
 Ire Kulgzss wragite
 Therolt iflerd lidfies thehe wed by
 Haurgolst (ion 20).


T638970 sthe rock the bHos, Vien canade 19  neccrdiners the the arrad wiccert bewhama
Brrtht sham the Ssemenilus eland. Is in ofky Al Thaca i is by Pesrisingituge sonetu, ar cong ald Kude.


Hugersi

Jalainge Wimmad


29< brck Mweresstseb